## Import HarmonicBalance

In [ ]:
using HarmonicBalance

## Problem definition

## We are analyzing two coupled parametrons, governed by
$$\ddot{x}_1+\omega_1^2(1-\lambda \cos(2\omega t))x_1+\gamma\dot{x}_1 + \alpha_1 x_1^3 - Jx_2=0\\
\ddot{x}_2+\omega_2^2(1-\lambda \cos(2\omega t))x_2+\gamma\dot{x}_2 + \alpha_2 x_2^3 - Jx_1=0$$

Define problem variables, parameters using the Symbolics.jl @variables macro. Time variables also need to be included

In [ ]:
@variables x, y, ω₁, ω₂,ω, γ,λ, t, T, ω, α₁, α₂, J;
@variables x(t), y(t);

Introduce equations to solve via DifferentialEquation input method

In [ ]:
natural_equation = [d(d(x, t),t) + ω₁^2*(1-λ*cos(2*ω*t)) * x + γ*d(x,t)  + α₁*x^3 - J*y , 
                    d(d(y,t),t)  + ω₂^2*(1-λ*cos(2*ω*t)) * y + γ*d(y,t)  + α₂*y^3 - J*x] 

dEOM_coupled = HarmonicBalance.DifferentialEquation(natural_equation, [x, y])

show(dEOM_coupled)

We haven't introduced harmonics yet. let's do that now

In [ ]:
HarmonicBalance.add_harmonic!(dEOM_coupled, x, ω) # x will rotate at ω
HarmonicBalance.add_harmonic!(dEOM_coupled, y, ω) # x will rotate at ω
show(dEOM_coupled)

Let's introduce our ansatz and let Harmonic Balance do the work for us

In [ ]:
@time averagedEOM = HarmonicBalance.get_harmonic_equations(dEOM_coupled, slow_time=T, fast_time=t)

![ChessUrl](https://c.tenor.com/VxSRLg7hSHMAAAAS/carlton-dance.gif "chess")

## Solving for steadystates

Let's solve for the slow-flow amplitudes $u1,u2,v1,v2$ in the steadystate. We will use homotopy continuation for that. In order to talk with the fantastic HomotopyContinuation.jl, we create a Problem instance. 

This holds the steady-state equations, and (optionally) the symbolic Jacobian which is needed for stability / linear response calculations. 

In [ ]:
problem_coupled = HarmonicBalance.Problem(averagedEOM)

Now the actual calculation: we analyze steadystates for varying external drive frequency, while fixing other parameters. We use ParameterRange and ParameterList types for that (basically ordered dictionaries)

In [ ]:
fixed_parameters = ParameterList([(ω₁, 1.),( ω₂, 1. ),(γ, 0.001 ),(α₁,  1.),(α₂, 1.),(λ,5E-2),(J,0.05)])
swept_parameters = ParameterRange(ω => LinRange(0.75,1.25, 100))

Get steady states!.

With random_warmup=True, we first solve the problem for some synthetic, complex parameters. These allow to filter out many solution paths that end up in $\infty$. After obtained solutions from this general complex system, we track the solutions to the real system for each parameter value.

In [ ]:
result = HarmonicBalance.get_steady_states(problem_coupled, swept_parameters, fixed_parameters; random_warmup=true, threading=false)

![ChessUrl](https://i.giphy.com/media/StWnlQipuBrz2/giphy.webp "chess")

## Plotting solutions

The most-basic plot we can do is checking the amplitudes $X_i=\sqrt{u_i^2+v_i^2}$. 

In [ ]:
HarmonicBalance.plot_1D_solutions(result, x="ω", y="sqrt(u1^2 + v1^2)", plot_only=["physical"])#,filename="plot_1D");
HarmonicBalance.plot_1D_solutions(result, x="ω", y="sqrt(u2^2 + v2^2)", plot_only=["physical"])#,filename="plot_1D");

A bit confusing. Let's look at one resonator only

In [ ]:
HarmonicBalance.plot_1D_solutions(result, x="ω", y="sqrt(u1^2 + v1^2)", plot_only=["physical","stable"])#,filename="plot_1D");

We can get insight on how the solutions where classified

In [ ]:
HarmonicBalance.plot_1D_jacobian_eigenvalues(result, x="ω", physical=true, stable=false)

You can save the output of the simulation to a .jld2 via save function, and load it via load

In [ ]:
HarmonicBalance.save("1D_sweep",result)
HarmonicBalance.load("1D_sweep.jld2")

## Two-dimensional parametric sweeps

Now let's assume we know relatively well what happens with an isolated parametron. We want to analyse how the frequency response changes because of the coupling to the other parametron. This type of problem is appropriate for a 2D sweep over $(\omega,J)$. 

We can simply add another parameter to ParameterRange and call the same get_steadystates function

It is convenient to sort the solutions for plotting. Nearest neighbor complex solutions are matched if sorting="naive"

In [ ]:
fixed_parameters = ParameterList([(ω₁, 1.),( ω₂, 1. ),(γ, 0.001 ),(α₁,  0.01),(α₂, 0.01),(λ,0.2)])
swept_parameters = ParameterRange(ω => LinRange(0.75,1.25, 100),J=>LinRange(0.,0.2,100));

@time result = HarmonicBalance.get_steady_states(problem_coupled, swept_parameters, fixed_parameters; random_warmup=true, threading=false,sorting="naive")

A simple, useful visualization is to see what's the number of solutions for each point of the 2D grid

In [ ]:
HarmonicBalance.plot_2D_phase_diagram(result, stable=false,observable="nsols")

Still, this plot is agnostic to some features of the bifurcation topology. Look at interactive versions

In [ ]:
HarmonicBalance.plot_2D_phase_diagram_interactive(result, stable=false, nrows=2, ncols=3,cut_dim="1",cut_type="solutions",observable="nsols")

In [ ]:
HarmonicBalance.plot_2D_phase_diagram_interactive(result, stable=false, nrows=2, ncols=2,cut_dim="1",cut_type="transform",observable="nsols",string_f=["sqrt(u1^2 + v1^2)","sqrt(u2^2 + v2^2)"])

In [ ]:
HarmonicBalance.plot_2D_phase_diagram_interactive(result, stable=false, nrows=5, ncols=6,cut_dim="1",cut_type="jacobian_eigenvalues",observable="nsols")

# Now it's turn turn! . Please pick one of the Exercises below

## Red pill exercise 

* Find the frequency conversion in a parametron with a cubic nonlinearity by analysing the solutions around half the parametric drive $\omega$ and $3\omega$. 

$$\ddot{x}+\omega_1^2(1-\lambda \cos(2\omega t))x+\gamma\dot{x} + \alpha x^3=0$$

* Then include a fifth order nonlinearity and include extra necessary harmonics. 
$$\ddot{x}+\omega_1^2(1-\lambda \cos(2\omega t))x+\gamma\dot{x} + \alpha x^3 + \beta x^5=0$$
How many harmonics do you need to reach convergence?

* Compare the number of solutions as a function of  drive frequency and frequency modulation parameter $\lambda$. Do you find a regime where new solutions appear?.

* Do the solutions converge as you add more tones?

## Blue pill exercise 

* Take three Kerr resonators with three different, non commensurable frequencies. 
* Add driving to one resonator and time-dependent couplings at the frequency difference of these modes, letting some nonzero phase $J_{ij}(t)=J\cos((\omega_i-\omega_j)t + \phi_{ij})$ (assume $\phi_{ij}=-\phi_{ji}$). 

$$\ddot{x}_1+\omega_1^2(1-\lambda \cos(2\omega t))x_1+\gamma\dot{x}_1 + \alpha_1 x_1^3 - J_{12}(t)x_2 - J_{13}(t)x_3=0\\
\ddot{x}_2+\omega_2^2(1-\lambda \cos(2\omega t))x_2+\gamma\dot{x}_2 + \alpha_2 x_2^3 - J_{21}(t)x_1 - J_{23}(t)x_3=0\\
\ddot{x}_3+\omega_3^2(1-\lambda \cos(2\omega t))x_3+\gamma\dot{x}_2 + \alpha_3 x_2^3 - J_{31}(t)x_1- J_{32}(t)x_2=0$$

* Find how solution amplitudes and linear fluctuation dynamics (Jacobian eigenvalues) depend on phases and nonlinearity when resonators form a ring or a chain. For it, analyse solutions vs coupling and phase
* Optional: Is there is any gauge symmetry in the problem? for example invariance with respect to some phase shift: $\phi_{12} \rightarrow \phi_{12} + \delta, \phi_{23}->\phi_{23} - \delta$).  Are symmetries broken by the nonlinear coefficient?